In [1]:
from azureml.train.hyperdrive import BayesianParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling
from azureml.train.hyperdrive import uniform, choice
from azureml.core import ScriptRunConfig, Experiment, Workspace
from azureml.core.experiment import Experiment
from sklearn.linear_model import LogisticRegression
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.widgets import RunDetails
import pandas
import sklearn

param_sampling = BayesianParameterSampling( {
        "--max_iter": uniform(10,30),
        "--C": choice(0.5, 1, 1.5)
    }
)   

In [2]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [3]:
import sklearn
print(sklearn.__version__)


1.5.1


In [4]:
    # Define a Bandit policy
policy = BanditPolicy(
    slack_factor=0.1,  # The slack factor to allow for some variability
    evaluation_interval=1,  # How often to evaluate the performance
    delay_evaluation=5  # Number of iterations to wait before evaluating
)

In [5]:
est = ScriptRunConfig(source_directory='.',  
                  script='train.py',  # Your training script
                  compute_target= 'mlprojcluster')



In [6]:

hyperdrive_config = HyperDriveConfig(
    run_config = est,  # Your script run configuration
    hyperparameter_sampling= param_sampling,
    policy=policy,  # Include the early stopping policy
    primary_metric_name='accuracy',  # Metric to optimize
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  # Goal to optimize
    max_total_runs=20,  # Total number of runs
    max_concurrent_runs=4, # Number of concurrent runs
)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


In [7]:
ws = Workspace.from_config('quick-starts-ws-278036')

In [8]:
exp = Experiment(ws,name='Hyper_exp2')

In [9]:
hdr = exp.submit(config=hyperdrive_config, show_output=True)

In [10]:
RunDetails(hdr).show()

2025-04-23 15:57:59.147674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-23 15:57:59.169251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-23 15:57:59.175869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-23 15:57:59.196459: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-23 15:58:00.224984: W tensorflow/comp

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

AttributeError: 'NoneType' object has no attribute 'get_metrics'